In [1]:
class Host(object):
    def __init__(self, key, lan):
        self.key = key
        self.lan = lan.key

    def get_lan(self):                          # Get Lan ID in Host
        return self.lan

In [2]:
class Lan(object):
    def __init__(self, key):
        self.key = key
        self.bridges = {}
        self.hosts = {}
        
        self.msgs = []
        self.db = key

    def add_bridge(self, bridge):               # Add Bridge IDs in Lan
        self.bridges[bridge.key] = bridge
        
    def add_host(self, host):                   # Add Hosts in Lan
        self.hosts[host.key] = host
        
    def send_msg(self):                         # Get Bridge IDs in Lan
        for msg in self.msgs:
            state = msg[0]
            sender = msg[1]
            for bridge in self.bridges:
                if(sender != bridge):
                    self.bridges[bridge].recieve_msg([state, self.key])
    
    def recieve_msg(self, msg):                  # Get Bridge IDs in Lan
        return self.msgs.append(msg)
    
    def update(self):
        self.msgs.sort(key=order)
        
        if len(self.msgs) != 0:
            best_config = self.msgs[0]
            self.root = best_config[0][0]
            self.d += int(best_config[0][1])

            lan_key = best_config[1]
            self.lans[lan_key][1] = "RP"

            self.state = [self.root, self.d, self.key]
        return self.state
        
    def get_connections(self):                  # Get Bridge IDs in Lan
        return self.bridges.keys()
    
    def get_hosts(self):                        # Get Host IDs in Lan
        return self.hosts.keys()

In [51]:
class Bridge(object):
    def __init__(self, key):
        self.key = key    
        self.root = key
        self.d = 0
        
        self.lans = {}
        self.state = [self.root, self.d, self.key]
        self.msgs = []

    def add_lan(self, lan):                     # Add Lan in Bridge
        self.lans[lan.key] = [lan,"DP"]

    def get_connections(self):                  # Get Lan IDs in Bridge
        return self.lans.keys()
    
    def port(self, lan_key):                    # Get Lan IDs in Bridge
        return [self.key, lan_key, self.lans[lan_key][1]]
    
    def status(self):                           # Get Bridge Status
        return self.state
    
    def send_msg(self):
        for lan in self.lans:
            self.lans[lan][0].recieve_msg([self.state, bridge])
        
    def recieve_msg(self, msg):                      # Receiver Bridge Status
        return self.msgs.append(msg)
    
    def update(self):
        self.msgs.sort(key=order)
        
        if len(self.msgs) != 0:
            best_config = self.msgs[0]
            self.root = best_config[0][0]
            self.d += int(best_config[0][1])

            lan_key = best_config[1]
            self.lans[lan_key][1] = "RP"

            self.state = [self.root, self.d, self.key]
        return self.state
        
    def is_root(self):
        return self.root == self.key

In [65]:
def order(msg):
    return int(msg[0][0][1])**2 + int(msg[0][1])**2 + int(msg[0][2][1])**2

In [52]:
class Network(object):
    def __init__(self):
        self.bridges = {}   # Bridges in Network
        self.lans = {}      # Lans in Network
        
    def add_bridge(self, bridge):                   # Add Bridge in Network
        self.bridges[bridge.key] = bridge
        
    def add_lan(self, lan):                         # Add Lan in Network
        self.lans[lan.key] = lan

    def add_port(self, lan_key, bridge_key):        # Add Port in resp Bridge and Lan
        if lan_key not in self.lans:
            self.add_lan(Lan(lan_key))
            
        if lan_key not in self.bridges[bridge_key].lans:
            self.bridges[bridge_key].add_lan(self.lans[lan_key])
            
        if bridge_key not in self.lans[lan_key].bridges:
            self.lans[lan_key].add_bridge(self.bridges[bridge_key])

    def get_bridges(self):                          # Get Bridges IDs in Network
        return self.bridges.keys()
    
    def get_lans(self):                             # Get Lans IDs in Network
        return self.lans.keys()

    def __iter__(self):
        return iter(self.bridges.values())

In [53]:
net = Network()

In [54]:
net.add_bridge(Bridge('B1'))
net.add_port('A','B1')
net.add_port('G','B1')
net.add_port('B','B1')

In [55]:
net.add_bridge(Bridge('B2'))
net.add_port('G','B2')
net.add_port('F','B2')

In [56]:
net.add_bridge(Bridge('B3'))
net.add_port('B','B3')
net.add_port('C','B3')

In [57]:
net.add_bridge(Bridge('B4'))
net.add_port('C','B4')
net.add_port('F','B4')
net.add_port('E','B4')

In [58]:
net.add_bridge(Bridge('B5'))
net.add_port('C','B5')
net.add_port('D','B5')
net.add_port('E','B5')

In [59]:
for bridge in net.bridges:
    print(net.bridges[bridge].status())

['B1', 0, 'B1']
['B2', 0, 'B2']
['B3', 0, 'B3']
['B4', 0, 'B4']
['B5', 0, 'B5']


In [60]:
for bridge in net.bridges:
    for lan in net.bridges[bridge].lans:
        print(net.bridges[bridge].port(lan))

['B1', 'A', 'DP']
['B1', 'G', 'DP']
['B1', 'B', 'DP']
['B2', 'G', 'DP']
['B2', 'F', 'DP']
['B3', 'B', 'DP']
['B3', 'C', 'DP']
['B4', 'C', 'DP']
['B4', 'F', 'DP']
['B4', 'E', 'DP']
['B5', 'C', 'DP']
['B5', 'D', 'DP']
['B5', 'E', 'DP']


In [61]:
net.lans['A'].send_msg()

In [62]:
net.lans['B'].send_msg()

In [63]:
print(net.bridges['B2'].msgs)

[]


In [64]:
print(net.lans['G'].msgs)

[]


In [66]:
print(net.bridges['B2'].update())

['B2', 0, 'B2']


In [67]:
net.bridges['B4'].send_msg()
net.bridges['B1'].send_msg()

In [68]:
net.lans['F'].send_msg()
net.lans['G'].send_msg()

In [69]:
print(net.bridges['B2'].msgs)

[[['B4', 0, 'B4'], 'F'], [['B1', 0, 'B1'], 'G']]


In [70]:
print(net.bridges['B2'].update())

['B1', 0, 'B2']


In [73]:
for lan in net.bridges['B2'].lans:
    print(net.bridges['B2'].port(lan))

['B2', 'G', 'RP']
['B2', 'F', 'DP']


In [74]:
net.lans['E'].msgs

[[['B4', 0, 'B4'], 'B5']]